In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from scenario import Scenario

import openpyxl as op

In [89]:
# Settings :

# from light_sim.ipynb
# Settings angles
#####
start_angle = -60
end_angle = 60
step_angle = 10
angles = np.arange(start_angle, end_angle + 1, step_angle)
# angles = [-60]
print(list(angles))
titres_colonnes = np.append(angles, np.array("temoin", dtype=object), axis=None)

# Annual parameters
###### # from light_sim.ipynb

######
# Settings, year and hours
start_date = pd.Timestamp('2023-01-01')  # Start of the year (avoid leap year)

input_path = "bdd_irr/step5_semitransp_HBmethod.xlsx"
output_path = "R_related/rapport_temoin_perso.csv"

[-60, -50, -40, -30, -20, -10, 0, 10, 20, 30, 40, 50, 60]


Import scenario :
1 = True = Backtracking (productible max)
0 = False = Effacement (irradiance au sol maximum)

In [90]:
## TESTS ##
# Instancier des objets scénarios
s_10pc = Scenario(file_path="scenarios/sc_10%.xlsx")
s_full_BT = Scenario(file_path="scenarios/sc_full_BT.xlsx")
# print(s1.scenario[:,:])

In [91]:
print(s_10pc)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


Import bdd_irr

In [92]:
df3 = pd.read_excel(input_path)
print(df3[0:24][:7])

  Month-Day-Hour  -60  -50  -40  -30  -20  -10  0  10  20  30  40  50  60  \
0       01-01-00    0    0    0    0    0    0  0   0   0   0   0   0   0   
1       01-01-01    0    0    0    0    0    0  0   0   0   0   0   0   0   
2       01-01-02    0    0    0    0    0    0  0   0   0   0   0   0   0   
3       01-01-03    0    0    0    0    0    0  0   0   0   0   0   0   0   
4       01-01-04    0    0    0    0    0    0  0   0   0   0   0   0   0   
5       01-01-05    0    0    0    0    0    0  0   0   0   0   0   0   0   
6       01-01-06    0    0    0    0    0    0  0   0   0   0   0   0   0   

   temoin  
0       0  
1       0  
2       0  
3       0  
4       0  
5       0  
6       0  


Création tab_1 : un tableau par scénario

In [93]:
n_rows = 8760

# Création d'un DataFrame avec des valeurs par défaut (par exemple, 0)
tab_1 = pd.DataFrame({
    'hoys': df3['Month-Day-Hour'],      #Hours of the Year
    'effacement': np.ones(n_rows),      #Panel priority mode, 1 = priority to photovoltaic production, 0 = panel positionned to let through max light
    'angle': np.zeros(n_rows),          #Angle of the panel
    'irradiance_sol': np.zeros(n_rows)  #Quantity of radiation received on the ground
})

Import PVSyst

In [94]:
# Chemin vers le fichier Excel
file_path = 'DATA/pvsyst_file.xlsm'

# Nom de l'onglet à lire
sheet_names = ['Tests','Illim']

# Lire l'onglet spécifique
pvsyst = pd.read_excel(file_path, sheet_name=sheet_names)

In [95]:
pvsyst_illim = pvsyst['Illim'].drop(index=list(range(12))).drop(columns=['Unnamed: 4', 'Unnamed: 5']).reset_index(drop=True)
pvsyst_illim.columns = ['Date','E_Grid','PR','PhiAng']
print(pvsyst_illim)

                     Date  E_Grid PR PhiAng
0     1990-01-01 00:00:00 -0.0132  0      0
1     1990-01-01 01:00:00 -0.0132  0      0
2     1990-01-01 02:00:00 -0.0132  0      0
3     1990-01-01 03:00:00 -0.0132  0      0
4     1990-01-01 04:00:00 -0.0132  0      0
...                   ...     ... ..    ...
8755  1990-12-31 19:00:00 -0.0132  0      0
8756  1990-12-31 20:00:00 -0.0132  0      0
8757  1990-12-31 21:00:00 -0.0132  0      0
8758  1990-12-31 22:00:00 -0.0132  0      0
8759  1990-12-31 23:00:00 -0.0132  0      0

[8760 rows x 4 columns]


In [96]:
pvsyst_tests = pvsyst['Tests'].drop(index=list(range(12))).drop(columns=pvsyst['Tests'].columns[4:]).reset_index(drop=True)
pvsyst_tests.columns = ['Date', 'PhiAng', 'E_Grid_SunTrack', 'Egrid Effectif']
print(pvsyst_tests)

                     Date PhiAng E_Grid_SunTrack Egrid Effectif
0     1990-01-01 00:00:00      0         -0.0132              1
1     1990-01-01 01:00:00      0         -0.0132              1
2     1990-01-01 02:00:00      0         -0.0132              1
3     1990-01-01 03:00:00      0         -0.0132              1
4     1990-01-01 04:00:00      0         -0.0132              1
...                   ...    ...             ...            ...
8755  1990-12-31 19:00:00      0         -0.0132             12
8756  1990-12-31 20:00:00      0         -0.0132             12
8757  1990-12-31 21:00:00      0         -0.0132             12
8758  1990-12-31 22:00:00      0         -0.0132             12
8759  1990-12-31 23:00:00      0         -0.0132             12

[8760 rows x 4 columns]


In [97]:
print(tab_1.head())
tab_1.iloc[1,0]

       hoys  effacement  angle  irradiance_sol
0  01-01-00         1.0    0.0             0.0
1  01-01-01         1.0    0.0             0.0
2  01-01-02         1.0    0.0             0.0
3  01-01-03         1.0    0.0             0.0
4  01-01-04         1.0    0.0             0.0


'01-01-01'

Remplir le tab_1 :

In [98]:
# Remplissage de la colonne 'effacement'
# Parcourir scénario, pour chaque case 0 (False), changer les 1 (panneaux en suntracking) de la colonne 'effacement' en 0 (panneaux effacés)

for i in range(s_10pc.scenario.shape[0]) :  # Pour chaque ligne
    for j in range(s_10pc.scenario.shape[1]) :  # Pour chaque colonne
        if s_10pc.scenario[i,j] == False :
            tab_1.loc[(tab_1['hoys'].str.slice(6, 8) == f'{j:02d}') & (tab_1['hoys'].str.slice(0, 2) == f'{i+1:02d}'), 'effacement'] = 0

In [99]:
# Remplissage de la colonne 'angle'
# Parcourir tableau_1, en fonction de 'effacement', remplir 'angle' et 'irradiance'
for i in range(tab_1.shape[0]) :
    if tab_1.at[i,'effacement'] == 0:
        #aller chercher irradiance au sol maximum dans tab_final (df3 dans ce .ipynb), et retrouver l'angle correspondant (nom de colonne)
        tab_1.loc[i,'irradiance_sol'] = df3.drop(columns=['Month-Day-Hour','temoin']).iloc[i].max() # Irradiance max
        tab_1.loc[i,'angle'] = df3.drop(columns=['Month-Day-Hour','temoin']).iloc[i].idxmax() # Colonne dont l'irradiance est max
        # /!\ Si égalité entre plusieurs angles, va choisir l'angle le moins élevé (-60 à priori)
    else: #'effacement' == 1
        # pvsyst_illim.loc[...] = angle PVSYST optimal, arrondir, puis trouver irradiance au sol correspondante
        tab_1.loc[i,'irradiance_sol'] = df3.at[i,str(round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle)]
        tab_1.loc[i,'angle'] = round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle

# Add control values to tab_1 :

tab_1['temoin'] = df3.loc[:,'temoin']
  

In [100]:
tab_1.to_excel("step5_semitransp_10%.xlsx", index=False, header=True)
tab_1

,hoys,effacement,angle,irradiance_sol,temoin
0,01-01-00,1.0,0.0,0.0,0
1,01-01-01,1.0,0.0,0.0,0
2,01-01-02,1.0,0.0,0.0,0
3,01-01-03,1.0,0.0,0.0,0
4,01-01-04,1.0,0.0,0.0,0
...,...,...,...,...,...
8755,12-31-19,1.0,0,0.0,0
8756,12-31-20,1.0,0,0.0,0
8757,12-31-21,1.0,0,0.0,0
8758,12-31-22,1.0,0,0.0,0


Calcul liste de taille 365 concernant rapport de irradiance_zone_témoin sur irradiance_zone_étude

In [4]:
# Chargement des données
file_path_new = 'step5_semitransp_10%.xlsx'
df_new = pd.read_excel(file_path_new)

# Extraction de la date à partir de la colonne 'hoys'
df_new['Date'] = df_new['hoys'].str[:5]
print(df_new.columns)
print(df_new)
# Agrégation journalière par sommation des colonnes 'irradiance_sol' et 'temoin'
daily_sum = df_new.groupby('Date')[['irradiance_sol', 'temoin']].sum().reset_index()

# Calcul des rapports journaliers
daily_sum['ratio'] = daily_sum['irradiance_sol'] / daily_sum['temoin']
daily_sum['ratio'] = daily_sum['ratio'].apply(lambda x: min(x, 1))

daily_sum['ratio'].to_csv("ratio.csv",index=False,header=False)

Index(['hoys', 'effacement', 'angle', 'irradiance_sol', 'temoin', 'Date'], dtype='object')
          hoys  effacement  angle  irradiance_sol  temoin   Date
0     01-01-00           1      0               0       0  01-01
1     01-01-01           1      0               0       0  01-01
2     01-01-02           1      0               0       0  01-01
3     01-01-03           1      0               0       0  01-01
4     01-01-04           1      0               0       0  01-01
...        ...         ...    ...             ...     ...    ...
8755  12-31-19           1      0               0       0  12-31
8756  12-31-20           1      0               0       0  12-31
8757  12-31-21           1      0               0       0  12-31
8758  12-31-22           1      0               0       0  12-31
8759  12-31-23           1      0               0       0  12-31

[8760 rows x 6 columns]
